In [ ]:
%load_ext autoreload

In [ ]:
%autoreload now

In [ ]:
from simple_rl.agents import QLearningAgent, SARSAAgent, ValueIterationAgent, DynaQAgent
from simple_rl.utils import plotting

In [ ]:
import multilevelsuccessoroptions.envs.tabular as envs
# import multilevelsuccessoroptions.utils.plotting as plotting
from multilevelsuccessoroptions.options.tabular.hierarchical_access_options import HAOS_Generator
from multilevelsuccessoroptions.successor_representation import SuccessorRepresentation

In [ ]:
from simpleoptions import PrimitiveOption

In [ ]:
import hdbscan
import numpy as np
from numpy.random import default_rng
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.pyplot as plt

In [ ]:
seed = 12345
rng = default_rng(seed)
env = envs.FourRooms(rng, exploration=False)

In [ ]:
primitives = [PrimitiveOption(a,env) for a in env.get_action_space()]
env.set_options(primitives)

# gamma = 0.99
# min_cluster_size = 9
# min_samples = 5

# sr = SuccessorRepresentation(env,gamma,0.2,True)
# model = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples, cluster_selection_method='eom',gen_min_span_tree=True, approx_min_span_tree=False,prediction_data=True, allow_single_cluster=False)
# gtr = HAOS_Generator(sr, model, rng, exploration_steps = 500_000, training_steps = 0, cut_off = 1_000, n_step_updates = True, multiprocessing = False, termination_factor = 1.0, alpha = 0.2, gamma = 1.0)
# gtr.generate_options(env)

In [ ]:
# termination_chances = gtr.options_dict["105_106"].termination_chance
# env = envs.FourRoomsStochasticTermination(rng, termination_chances, False, 0.0)

In [ ]:
# plotting.plot_maze_env(env);

In [ ]:
actions = env.get_action_space()

In [ ]:
q_agent = QLearningAgent(rng, actions, 0.99, 0.4, 0.2, 1.0, 0.0)
q_agent.run(env, int(1e6))

In [ ]:
plotting.plot_agent_policy(q_agent,env)

In [ ]:
sarsa_agent = SARSAAgent(rng, actions, 0.99, 0.4, 0.2, 1.0, 0.0)
sarsa_agent.run(env,int(1e6))

In [ ]:
plotting.plot_agent_policy(sarsa_agent,env)

In [ ]:
value_iteration_agent = ValueIterationAgent(rng, actions, 0.99, 1e-15, 1.0)
value_iteration_agent.run(env,None)

In [ ]:
plotting.plot_agent_policy(value_iteration_agent,env)

In [ ]:
num_layers = 2
ratio_sum = sum([(n + 1) / (num_layers + 1) for n in range(num_layers + 1)])
p = [(n + 1) / ((num_layers + 1) * ratio_sum) for n in range(num_layers + 1)]
print(p)

In [ ]:
dyna_agent = DynaQAgent(rng, actions, 0.99, 0.2, 0.2, 10, 1.0, 0.0)
dyna_agent.run(env, 500)

In [ ]:
plotting.plot_agent_policy(dyna_agent,env)
